In [1]:
from recognition import evaluate_model as em
from recognition import csv_utils as cu


Using TensorFlow backend.


# 获取类到索引的字典

In [2]:
class_indices = cu.get_class_indices('class_indices.csv')
print(class_indices)

{'且': 0, '世': 1, '东': 2, '九': 3, '亭': 4, '今': 5, '从': 6, '令': 7, '作': 8, '使': 9, '侯': 10, '元': 11, '光': 12, '利': 13, '印': 14, '去': 15, '受': 16, '右': 17, '司': 18, '合': 19, '名': 20, '周': 21, '命': 22, '和': 23, '唯': 24, '堂': 25, '士': 26, '多': 27, '夜': 28, '奉': 29, '女': 30, '好': 31, '始': 32, '字': 33, '孝': 34, '守': 35, '宗': 36, '官': 37, '定': 38, '宜': 39, '室': 40, '家': 41, '寒': 42, '左': 43, '常': 44, '建': 45, '徐': 46, '御': 47, '必': 48, '思': 49, '意': 50, '我': 51, '敬': 52, '新': 53, '易': 54, '春': 55, '更': 56, '朝': 57, '李': 58, '来': 59, '林': 60, '正': 61, '武': 62, '氏': 63, '永': 64, '流': 65, '海': 66, '深': 67, '清': 68, '游': 69, '父': 70, '物': 71, '玉': 72, '用': 73, '申': 74, '白': 75, '皇': 76, '益': 77, '福': 78, '秋': 79, '立': 80, '章': 81, '老': 82, '臣': 83, '良': 84, '莫': 85, '虎': 86, '衣': 87, '西': 88, '起': 89, '足': 90, '身': 91, '通': 92, '遂': 93, '重': 94, '陵': 95, '雨': 96, '高': 97, '黄': 98, '鼎': 99}


# 获取模型的信息

In [3]:
models_list = [
    'models/VGG16/model',
    'models/ResNet50/model'
]

# 评估网络在验证集上的性能

### 获取所有模型对应的数据生成器

In [4]:
valid_generators, valid_predictions = em.get_generators_and_predictions_for_models_list(models_list, 
                                                                                        'datasets/validation',
                                                                                        batch_process=256,
                                                                                        num_classes=100)

Obtaining the generator of the model 'models/VGG16/model', the model's information are as follows: 
{'batch_size': '128', 'color_mode': 'grayscale', 'image_size': '96', 'rescale': '1./255', 'algorithm': 'ke_aug', 'horizontal_flip': 'False', 'vertical_flip': 'False', 'width_shift_range': '0.1', 'height_shift_range': '0.1', 'rotation_range': '10', 'shear_range': '10.0', 'zoom_range': '0.1', 'channel_shift_range': '0.0', 'shuffle': 'False', 'augment_size': '6'}
Found 4000 images belonging to 100 classes.
Obtaining the generator of the model 'models/ResNet50/model', the model's information are as follows: 
{'batch_size': '32', 'color_mode': 'grayscale', 'image_size': '224', 'rescale': '1./255', 'algorithm': 'ke_aug', 'horizontal_flip': 'False', 'vertical_flip': 'False', 'width_shift_range': '0.1', 'height_shift_range': '0.1', 'rotation_range': '10', 'shear_range': '10.0', 'zoom_range': '0.1', 'channel_shift_range': '0.0', 'shuffle': 'False', 'augment_size': '6'}
Found 4000 images belonging

### 检查所有模型在验证集上的性能

In [5]:
print('Validation accuracy for each model:')
for model_info, generator, prediction in zip(models_list, valid_generators, valid_predictions):
    print(model_info)
    for k in {1, 5}:
        _, _, _, _, _ = em.evaluate_topk_accuracy_by_single_generator(generator, 
                                                                      class_indices,
                                                                      prediction,
                                                                      k=k)

Validation accuracy for each model:
models/VGG16/model
Top-1 accuracy: 83.67%
Top-5 accuracy: 94.42%
models/ResNet50/model
Top-1 accuracy: 86.65%
Top-5 accuracy: 96.25%


### 评估模型集成后的性能

In [6]:
print('Validation accuracy with model ensemble: ')
for k in {1, 5}:
    _, ensemble_topk_indices = em.predictions_ensemble(valid_generators, 
                                                       valid_predictions, 
                                                       algorithm='mean', 
                                                       k=k)
    _, _, _, _, _ = em.evaluate_topk_accuracy_by_single_generator(valid_generators[0], 
                                                                  class_indices,
                                                                  topk_indices=ensemble_topk_indices,
                                                                  k=k)

Validation accuracy with model ensemble: 
Top-1 accuracy: 88.98%
Top-5 accuracy: 96.43%


# 获取在测试集上模型集成后的结果

### 获取所有模型的生成器和预测值
- 如果你没有备份所有模型的预测值，你可以运行函数 `get_generators_and_predictions_for_models_list` 来获取模型对应的生成器和预测值，在这之后，你可以使用函数 `save_predictions_for_models_list` 来保存预测值
- 如果你已经备份了所有模型的预测值，你应该运行函数`get_generators_for_models_list` 和 `get_predictions_from_backup_for_models_list`

In [7]:
test_generators, test_predictions = em.get_generators_and_predictions_for_models_list(models_list, 
                                                                                      'datasets/validation',
                                                                                      batch_process=256,
                                                                                      num_classes=100)

Obtaining the generator of the model 'models/VGG16/model', the model's information are as follows: 
{'batch_size': '128', 'color_mode': 'grayscale', 'image_size': '96', 'rescale': '1./255', 'algorithm': 'ke_aug', 'horizontal_flip': 'False', 'vertical_flip': 'False', 'width_shift_range': '0.1', 'height_shift_range': '0.1', 'rotation_range': '10', 'shear_range': '10.0', 'zoom_range': '0.1', 'channel_shift_range': '0.0', 'shuffle': 'False', 'augment_size': '6'}
Found 4000 images belonging to 100 classes.
Obtaining the generator of the model 'models/ResNet50/model', the model's information are as follows: 
{'batch_size': '32', 'color_mode': 'grayscale', 'image_size': '224', 'rescale': '1./255', 'algorithm': 'ke_aug', 'horizontal_flip': 'False', 'vertical_flip': 'False', 'width_shift_range': '0.1', 'height_shift_range': '0.1', 'rotation_range': '10', 'shear_range': '10.0', 'zoom_range': '0.1', 'channel_shift_range': '0.0', 'shuffle': 'False', 'augment_size': '6'}
Found 4000 images belonging

In [8]:
em.save_predictions_for_models_list(models_list, test_predictions, 'predictions')

Saving the prediction in the file 'predictions/VGG16_model.npy'
Saving the prediction in the file 'predictions/ResNet50_model.npy'
All the preditions have been saved!


In [9]:
# get all generators by models' parameters
_, test_generators = em.get_generators_for_models_list(models_list, 'datasets/validation')
# get all the predictions 
test_predictions = em.get_predictions_from_backup_for_models_list(models_list, 'predictions')

Obtaining the generator of the model 'models/VGG16/model', the model's information are as follows: 
{'batch_size': '128', 'color_mode': 'grayscale', 'image_size': '96', 'rescale': '1./255', 'algorithm': 'ke_aug', 'horizontal_flip': 'False', 'vertical_flip': 'False', 'width_shift_range': '0.1', 'height_shift_range': '0.1', 'rotation_range': '10', 'shear_range': '10.0', 'zoom_range': '0.1', 'channel_shift_range': '0.0', 'shuffle': 'False', 'augment_size': '6'}
Found 4000 images belonging to 100 classes.
Obtaining the generator of the model 'models/ResNet50/model', the model's information are as follows: 
{'batch_size': '32', 'color_mode': 'grayscale', 'image_size': '224', 'rescale': '1./255', 'algorithm': 'ke_aug', 'horizontal_flip': 'False', 'vertical_flip': 'False', 'width_shift_range': '0.1', 'height_shift_range': '0.1', 'rotation_range': '10', 'shear_range': '10.0', 'zoom_range': '0.1', 'channel_shift_range': '0.0', 'shuffle': 'False', 'augment_size': '6'}
Found 4000 images belonging

### 获取top-k类别的哈希值

In [10]:
_, ensemble_topk_indices = em.predictions_ensemble(test_generators, 
                                                   test_predictions, 
                                                   algorithm='mean', 
                                                   k=5)

# 将预测答案写入文件

In [12]:
results = cu.get_topk_pd_data(test_generators[0], 
                         class_indices, 
                         topk_indices=ensemble_topk_indices)
cu.write_pd_data_into_csv(results, 
                          'top5_results.csv', 
                          have_chinese=True, 
                          index=None, 
                          header=True)
print('OK!')

OK!


# 获取top-1的结果并写入文件

In [13]:
top1_results = cu.get_top1_pd_data_from_topk('top5_results.csv', 
                                             'top1_results.csv')
cu.write_pd_data_into_csv(results, 
                          'top1_results.csv', 
                          have_chinese=True, 
                          index=None, 
                          header=True)
print('OK!')

OK!
